In [1]:
import requests
import json
import urllib3

# Disable SSL warnings for self-signed Proxmox certificates
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Proxmox API Credentials
PROXMOX_HOST = "192.168.0.100"  # Replace with your Proxmox server IP
USERNAME = "root@pam"  # Proxmox username
PASSWORD = "denpen7172"  # Proxmox password
NODE = "proxmox"  # Your Proxmox node name
VM_ID = 100  # The VM ID you want to monitor

# Authenticate and get a session ticket/token
def get_proxmox_ticket():
    url = f"https://{PROXMOX_HOST}:8006/api2/json/access/ticket"
    payload = {"username": USERNAME, "password": PASSWORD}
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    response = requests.post(url, data=payload, headers=headers, verify=False)

    if response.status_code == 200:
        data = response.json()
        return data["data"]["ticket"], data["data"]["CSRFPreventionToken"]
    else:
        print(f"❌ Failed to authenticate: {response.text}")
        return None, None

# Fetch VM usage statistics from Proxmox API
def get_vm_usage(vm_id, ticket):
    url = f"https://{PROXMOX_HOST}:8006/api2/json/nodes/{NODE}/qemu/{vm_id}/status/current"
    headers = {"Cookie": f"PVEAuthCookie={ticket}"}

    response = requests.get(url, headers=headers, verify=False)

    if response.status_code == 200:
        data = response.json()
        if "data" in data:
            print("✅ API Response:", json.dumps(data["data"], indent=4))  # Print full response
            return data["data"]
        else:
            print(f"❌ No usage data found for VM {vm_id}.")
            print("🔍 API Response:", response.text)
            return None
    else:
        print(f"❌ Failed to fetch usage for VM {vm_id}. HTTP Status: {response.status_code}")
        print("🔍 Response:", response.text)
        return None

# Main execution
if __name__ == "__main__":
    ticket, csrf_token = get_proxmox_ticket()
    if ticket:
        vm_data = get_vm_usage(VM_ID, ticket)
        if vm_data:
            print("\n✅ Successfully retrieved VM usage stats!")
            print(f"🔹 VM Name: {vm_data.get('name', 'Unknown')}")
            print(f"🔹 Status: {vm_data.get('status', 'Unknown')}")
            print(f"🔹 Uptime: {vm_data.get('uptime', 'Unknown')} seconds")
            print(f"🔹 CPU Cores: {vm_data.get('cpus', 'Unknown')}")
            print(f"🔹 Max Memory: {round(vm_data.get('maxmem', 0) / (1024 * 1024), 2)} MB")
            print(f"🔹 Network In: {round(vm_data.get('netin', 0) / 1024, 2)} KB")
            print(f"🔹 Network Out: {round(vm_data.get('netout', 0) / 1024, 2)} KB")
            print(f"🔹 Disk Read: {round(vm_data.get('diskread', 0) / (1024 * 1024), 2)} MB")
            print(f"🔹 Disk Write: {round(vm_data.get('diskwrite', 0) / (1024 * 1024), 2)} MB")
        else:
            print("❌ Could not retrieve VM usage data.")
    else:
        print("❌ Authentication failed. Exiting script.")


✅ API Response: {
    "cpus": 2,
    "running-qemu": "9.0.2",
    "vmid": 100,
    "name": "rhel8-vm",
    "serial": 1,
    "freemem": 3382611968,
    "disk": 0,
    "qmpstatus": "running",
    "maxmem": 4294967296,
    "uptime": 1182,
    "status": "running",
    "pid": 104400,
    "agent": 1,
    "blockstat": {
        "pflash0": {
            "zone_append_operations": 0,
            "invalid_zone_append_operations": 0,
            "unmap_operations": 0,
            "failed_rd_operations": 0,
            "invalid_rd_operations": 0,
            "unmap_merged": 0,
            "wr_operations": 0,
            "rd_merged": 0,
            "flush_operations": 0,
            "account_failed": true,
            "wr_total_time_ns": 0,
            "zone_append_merged": 0,
            "wr_bytes": 0,
            "unmap_bytes": 0,
            "failed_zone_append_operations": 0,
            "wr_highest_offset": 0,
            "rd_total_time_ns": 0,
            "account_invalid": true,
            "